In [1]:
# Note: you need to be using OpenAI Python v0.27.0 for the code below to work
import os
import openai
import pandas as pd
import nltk

from sklearn.utils import shuffle
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer
from tqdm import tqdm

nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/gabriellee/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/gabriellee/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [86]:
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

In [6]:
# !pip -q install -U -q PyDrive
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials
# auth.authenticate_user()  #click allow access
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

In [20]:
os.environ["OPENAI_API_KEY"] = "<YOUR_API_HERE>"
openai.api_key = os.getenv("OPENAI_API_KEY")

In [8]:
# id = "1th1mkhwZ5YvKEru9po8k0lxxD_BJ9Xqr"
# downloaded = drive.CreateFile({'id':id}) 
# downloaded.GetContentFile('balanced_test.csv')  

In [5]:
train = pd.read_csv('fulltrain.csv', header = None)
train.columns = ['label', 'text']
train

,label,text
0,1,"A little less than a decade ago, hockey fans w..."
1,1,The writers of the HBO series The Sopranos too...
2,1,Despite claims from the TV news outlet to offe...
3,1,After receiving 'subpar' service and experienc...
4,1,After watching his beloved Seattle Mariners pr...
...,...,...
48849,4,The ruling Kuomintang (KMT) has claimed owners...
48850,4,The Taipei city government has encouraged the ...
48851,4,President Ma Ying-jeou said Friday that a park...
48852,4,The families of the four people who were kille...


In [2]:
test = pd.read_csv('balanced_test.csv', header = None)
test.columns = ['label', 'text']
test

,label,text
0,1,When so many actors seem content to churn out ...
1,1,In what football insiders are calling an unex...
2,1,In a freak accident following Game 3 of the N....
3,1,North Koreas official news agency announced to...
4,1,The former Alaska Governor Sarah Palin would b...
...,...,...
2995,4,The Air Force mistakenly gave rival companies ...
2996,4,The United Nations climate chief on Friday cha...
2997,4,River Plate midfielder Diego Buonanotte has un...
2998,4,Lawmakers were on the brink Tuesday of exempti...


In [31]:
model_id = 'gpt-3.5-turbo'

def get_gpt_reply(m):
    response = openai.ChatCompletion.create(
        model=model_id,
        messages=m
    )
    m.append({'role': response.choices[0].message.role, 'content': response.choices[0].message.content})
    return m

messages = []
messages.append({'role': 'system', 'content': 'How may I help you?'})
messages = get_gpt_reply(messages)
print('{0}: {1}\n'.format(messages[-1]['role'].strip(), messages[-1]['content'].strip()))

assistant: As an AI language model, I can assist you with a wide range of tasks such as answering questions, providing information on various topics, generating ideas, helping you with language translations, making calculations, and more. Feel free to ask me anything and I will do my best to provide a helpful response.



In [42]:
responses = list()
pbar = tqdm(total=3000)

for index, row in test.iterrows():
    this_message = messages.copy()
    this_message.append({'role': 'user', 'content': 'Decide whether the following news is a satire, hoax, propaganda or reliable news, reply only with its classification, without justification. \n'})
    this_message = get_gpt_reply(this_message)

    prompt = "News: " + row['text']
    this_message.append({'role': 'user', 'content': prompt})
    try:
        output = get_gpt_reply(this_message)
        responses.append(output[-1]['content'].strip())
    except Exception:
        responses.append('NaN')
    
    pbar.update()

pbar.close()


 14%|█████▎                                | 171/1209 [28:19<2:51:53,  9.94s/it]

 19%|███████▍                                | 193/1038 [10:04<46:53,  3.33s/it]


 37%|██████████████▉                         | 387/1038 [20:24<30:52,  2.85s/it]


 56%|██████████████████████▍                 | 581/1038 [29:30<21:57,  2.88s/it]


 75%|█████████████████████████████▊          | 775/1038 [42:50<11:13,  2.56s/it]


 93%|█████████████████████████████████████▎  | 969/1038 [51:48<02:46,  2.42s/it]


100%|███████████████████████████████████████| 1038/1038 [54:55<00:00,  3.17s/it]


In [ ]:
test['responses'] = responses

In [ ]:
test.to_csv('result_test_raw.csv', encoding = 'utf-8-sig') 

In [15]:
results = pd.read_csv('result_test_raw.csv')
results = results[['text', 'label', 'raw_pred']]
results

,text,label,raw_pred
0,When so many actors seem content to churn out ...,1,Satire.
1,In what football insiders are calling an unex...,1,Satire.
2,In a freak accident following Game 3 of the N....,1,The news article appears to be a satire.
3,North Koreas official news agency announced to...,1,Satire.
4,The former Alaska Governor Sarah Palin would b...,1,Satire.
...,...,...,...
2995,The Air Force mistakenly gave rival companies ...,4,This is a reliable news article.
2996,The United Nations climate chief on Friday cha...,4,Reliable News.
2997,River Plate midfielder Diego Buonanotte has un...,4,This is a reliable news.
2998,Lawmakers were on the brink Tuesday of exempti...,4,This news appears to be reliable news.


In [22]:
na_indices = results[results['raw_pred'].isna()].index

In [24]:
na_indices

Int64Index([756, 822, 854, 1179, 1301, 1379, 1865, 2015, 2079, 2216, 2257,
            2471, 2707],
           dtype='int64')

In [21]:
model_id = 'gpt-3.5-turbo'

def get_gpt_reply(m):
    response = openai.ChatCompletion.create(
        model=model_id,
        messages=m
    )
    m.append({'role': response.choices[0].message.role, 'content': response.choices[0].message.content})
    return m

messages = []
messages.append({'role': 'system', 'content': 'How may I help you?'})
messages = get_gpt_reply(messages)
print('{0}: {1}\n'.format(messages[-1]['role'].strip(), messages[-1]['content'].strip()))

assistant: Hello! As an AI language model, I can assist you with a variety of tasks. You can ask me for information, help with a project, personal advice, or just have a casual conversation. So, how can I be of assistance?



In [27]:
na_results = results.iloc[na_indices]
na_results

,text,label,raw_pred
756,Freed Slaves In Washington? Dont Blame Us We W...,2,NaN
822,"Mom complains about Bibles in her sons school,...",2,NaN
854,"Rehana, The Famous Female Kurdish Fighter Resp...",2,NaN
1179,11 Year Old Boy Being Changed Into Girl In Cal...,2,NaN
1301,What Liberals Demand Parents Do This Christmas...,2,NaN
1379,"Michael Browns Mother, Grandmother And Relativ...",2,NaN
1865,With a massive backlash now mounting against ...,3,NaN
2015,Jared Diamond is no doom-and-gloomer; he's a ...,3,NaN
2079,Prepare to have your world rocked. What you'r...,3,NaN
2216,U.S. Senator Charles Grassley of Iowa has con...,3,NaN


In [41]:
def reduce_text(text):
    tokens = word_tokenize(text)
    reduced_tokens = tokens[:3000]
    return ' '.join(reduced_tokens)

In [39]:
for index, row in na_results.iterrows():
    print(len(word_tokenize(reduce_text(row['text']))))

3500
3501
3500
3500
3500
3500
3501
1942
3500
682
3500
3500
991


In [42]:

na_responses = list()
pbar = tqdm(total=13)

for index, row in na_results.iterrows():
    this_message = messages.copy()
    this_message.append({'role': 'user', 'content': 'Decide whether the following news is a satire, hoax, propaganda or reliable news, reply only with its classification, without justification. \n'})
    this_message = get_gpt_reply(this_message)

    prompt = "News: " + reduce_text(row['text'])
    this_message.append({'role': 'user', 'content': prompt})
    
    output = get_gpt_reply(this_message)
    na_responses.append(output[-1]['content'].strip())
    
    pbar.update()

pbar.close()

100%|███████████████████████████████████████████| 13/13 [00:34<00:00,  2.68s/it]


In [64]:
results.at[na_indices, 'raw_pred'] = na_responses

In [90]:
def get_pred(text):
    lower_text = text.lower()
    tokens = list(map(lambda t: stemmer.stem(t), word_tokenize(lower_text)))
    pred = list()
    if 'satire' in lower_text or stemmer.stem('satire') in tokens:
        pred.append(1)
    if 'hoax' in lower_text or stemmer.stem('hoax') in tokens:
        pred.append(2)
    if 'propaganda' in lower_text or stemmer.stem('propaganda') in tokens:
        pred.append(3)
    if ('reliable' in lower_text or stemmer.stem('reliable') in tokens) and 'unreliable' not in lower_text:
        pred.append(4)
    
    if len(pred) != 1:
        return 'NaN'
    return pred[0]

In [91]:
results['pred'] = results['raw_pred'].apply(get_pred)

In [110]:
cleaned_results = results[results['pred'] != 'NaN']
cleaned_results

,text,label,raw_pred,pred
0,When so many actors seem content to churn out ...,1,Satire.,1
1,In what football insiders are calling an unex...,1,Satire.,1
2,In a freak accident following Game 3 of the N....,1,The news article appears to be a satire.,1
3,North Koreas official news agency announced to...,1,Satire.,1
4,The former Alaska Governor Sarah Palin would b...,1,Satire.,1
...,...,...,...,...
2995,The Air Force mistakenly gave rival companies ...,4,This is a reliable news article.,4
2996,The United Nations climate chief on Friday cha...,4,Reliable News.,4
2997,River Plate midfielder Diego Buonanotte has un...,4,This is a reliable news.,4
2998,Lawmakers were on the brink Tuesday of exempti...,4,This news appears to be reliable news.,4


In [87]:
stemmer.stem('satirical')

'satir'

In [88]:
stemmer.stem('satire')

'satir'

In [111]:
cleaned_results.to_csv('result_test.csv', index=False)

## Results analysis

In [113]:
test_pred = pd.read_csv('result_test.csv')
test_pred

,text,label,raw_pred,pred
0,When so many actors seem content to churn out ...,1,Satire.,1
1,In what football insiders are calling an unex...,1,Satire.,1
2,In a freak accident following Game 3 of the N....,1,The news article appears to be a satire.,1
3,North Koreas official news agency announced to...,1,Satire.,1
4,The former Alaska Governor Sarah Palin would b...,1,Satire.,1
...,...,...,...,...
2867,The Air Force mistakenly gave rival companies ...,4,This is a reliable news article.,4
2868,The United Nations climate chief on Friday cha...,4,Reliable News.,4
2869,River Plate midfielder Diego Buonanotte has un...,4,This is a reliable news.,4
2870,Lawmakers were on the brink Tuesday of exempti...,4,This news appears to be reliable news.,4


In [114]:
from sklearn.metrics import classification_report

In [126]:
print(classification_report(test_pred['label'], test_pred['pred']))

              precision    recall  f1-score   support

           1       0.88      0.98      0.93       749
           2       0.73      0.02      0.03       697
           3       0.37      0.40      0.39       705
           4       0.56      0.99      0.72       721

    accuracy                           0.61      2872
   macro avg       0.64      0.60      0.52      2872
weighted avg       0.64      0.61      0.52      2872

